In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import sys
import json
from infer_functions import *


In [117]:
modules_to_reload = ["infer_functions"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

Data formation

In [119]:
static_tensor_dataset = create_tensor_dataset("static", 4, limit=100, categories= 9)

Catting AAPL.csv
Catting ABBV.csv
Catting ABT.csv
Catting ACN.csv
Catting ADBE.csv
Catting AMAT.csv
Catting AMD.csv
Catting AMGN.csv
Catting AMZN.csv
Catting AVGO.csv
Catting BA.csv
Catting BKNG.csv
Catting CAT.csv
Catting CMCSA.csv
Catting COP.csv
Catting COST.csv
Catting CRM.csv
Catting CSCO.csv
Catting CVX.csv
Catting DHR.csv
Catting DIS.csv
Catting HD.csv
Catting HON.csv
Catting IBM.csv
Catting INTC.csv
Catting INTU.csv
Catting JNJ.csv
Catting KO.csv
Catting LIN.csv
Catting LLY.csv
Catting LOW.csv
Catting MA.csv
Catting MCD.csv
Catting MRK.csv
Catting MSFT.csv
Catting NKE.csv
Catting NOW.csv
Catting NVDA.csv
Catting ORCL.csv
Catting PEP.csv
Catting PFE.csv
Catting PG.csv
Catting PM.csv
Catting QCOM.csv
Catting RTX.csv
Catting SPGI.csv
Catting TMO.csv
Catting TMUS.csv
Catting TSLA.csv
Catting TXN.csv
Catting UBER.csv
Catting UNP.csv
Catting UPS.csv
Catting V.csv
Catting VZ.csv
Catting WMT.csv
Catting XOM.csv


In [149]:
dynamic_tensor_dataset = create_tensor_dataset("dynamic", 4, limit=100, categories= 9)

Catting AAPL.csv
Catting ABBV.csv
Catting ABT.csv
Catting ACN.csv
Catting ADBE.csv
Catting AMAT.csv
Catting AMD.csv
Catting AMGN.csv
Catting AMZN.csv
Catting AVGO.csv
Catting BA.csv
Catting BKNG.csv
Catting BLK.csv
Catting BX.csv
Catting CAT.csv
Catting COP.csv
Catting CRM.csv
Catting CSCO.csv
Catting CVX.csv
Catting DHR.csv
Catting DIS.csv
Catting GE.csv
Catting HD.csv
Catting HON.csv
Catting IBM.csv
Catting INTC.csv
Catting INTU.csv
Catting JNJ.csv
Catting KO.csv
Catting LIN.csv
Catting LLY.csv
Catting LOW.csv
Catting MA.csv
Catting MCD.csv
Catting META.csv
Catting MRK.csv
Catting MSFT.csv
Catting NFLX.csv
Catting NOW.csv
Catting NVDA.csv
Catting ORCL.csv
Catting PEP.csv
Catting PFE.csv
Catting PG.csv
Catting PLD.csv
Catting PM.csv
Catting QCOM.csv
Catting RTX.csv
Catting SPGI.csv
Catting TMO.csv
Catting TMUS.csv
Catting TSLA.csv
Catting TXN.csv
Catting UBER.csv
Catting UNH.csv
Catting UNP.csv
Catting VZ.csv
Catting WMT.csv


In [127]:
print(len(dynamic_tensor_dataset))
for i, thing in list(enumerate(dynamic_tensor_dataset))[:100]:
    print(thing)

1977
((tensor([[1.5255e-05, 6.9777e-06, 1.1259e-05, 3.5437e-05, 1.7395e-05, 1.7972e-05,
         1.6365e-05, 2.3616e-05],
        [1.5761e-05, 7.0899e-06, 1.3775e-05, 6.9498e-05, 2.2771e-05, 2.5506e-05,
         1.8180e-05, 8.8955e-06],
        [2.0827e-05, 7.2160e-06, 1.7574e-05, 3.0669e-05, 3.3413e-05, 3.4662e-05,
         2.4180e-05, 4.4263e-05],
        [1.5593e-05, 4.6963e-06, 1.0889e-05, 1.3764e-05, 1.7267e-05, 1.6415e-05,
         1.6350e-05, 4.9416e-05]], dtype=torch.float64), tensor([0.0000e+00, 1.0000e+00, 4.0000e+00, 1.2730e+11], dtype=torch.float64)), tensor([0., 0., 1., 0.], dtype=torch.float64))
((tensor([[1.7572e-05, 6.9777e-06, 1.9013e-05, 2.7140e-05, 3.5838e-05, 2.3573e-05,
         2.2302e-05, 3.0932e-05],
        [1.5537e-05, 7.0899e-06, 1.1481e-05, 3.6000e-05, 1.7275e-05, 1.8358e-05,
         1.6685e-05, 2.4297e-05],
        [1.6040e-05, 7.2160e-06, 1.4142e-05, 7.1264e-05, 2.4004e-05, 2.5966e-05,
         1.8489e-05, 9.1133e-06],
        [2.1213e-05, 7.3539e-06, 1.7

In [155]:
with open(r"..\other_pickle\measures.json", "r") as file:
    measures = json.load(file)

static_size = len(measures["static"])
dynamic_size = len(measures["dynamic"])

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

STATIC_BATCH_SIZE = 20
DYNAMIC_BATCH_SIZE = 2

static_train_dataloader = torch.utils.data.DataLoader(static_tensor_dataset, batch_size=STATIC_BATCH_SIZE, shuffle=True)
dynamic_train_dataloader = torch.utils.data.DataLoader(dynamic_tensor_dataset, batch_size=DYNAMIC_BATCH_SIZE, shuffle=False)

cpu


Static model

In [122]:
HIDDEN_SIZE = 200
LAYERS = 8

class StaticLSTM(nn.Module):

    def __init__(self, hidden_dim, batch_size, layers, input, categories=0):
        super(StaticLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.layers_num = layers
        
        #input is all the embedding vectors plus all the other variables
        self.lstm = nn.LSTM(input, hidden_dim, num_layers=layers, batch_first=True) 
        self.hidden = (torch.zeros(layers,batch_size,hidden_dim),torch.zeros(layers,batch_size,hidden_dim))
        
        #Squeeeze them into 1 dimension
        if categories > 0:
            self.hidden2label = nn.Linear(hidden_dim, categories)
        else:
            self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, batch_tensor):
        lstm_out, self.hidden = self.lstm(batch_tensor)
        last_timestep_output = lstm_out[:, -1, :]
        sales = self.hidden2label(last_timestep_output)
        return sales
    
    def hidden_reset(self):
        #reset the hidden and cell state after each epoch
        self.hidden = (torch.zeros(self.layers_num,self.batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,self.batch_size,self.hidden_dim))
    def batch_reset(self,batch_size):
        self.hidden = (torch.zeros(self.layers_num,batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,batch_size,self.hidden_dim))
    def flatten_parameters(self):
        self.lstm.flatten_parameters()

static_model = StaticLSTM(HIDDEN_SIZE, STATIC_BATCH_SIZE, LAYERS, static_size, categories=6)
static_model = static_model.double()

In [94]:
loss_function = nn.L1Loss()
loss_scores = [float('inf')]
learning_rate = 0.01
epochs = 2
static_model = static_model.to(device)
optimizer = optim.Adam(static_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    static_model.hidden_reset()
    epoch_loss = 0
    for i, batch in enumerate(static_train_dataloader):
        (input, worthless_input), label = batch
        if input.shape[0] != STATIC_BATCH_SIZE:
            static_model.batch_reset(input.shape[0])
            print("Reset triggered due to batch size mismatch")

        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = static_model(input).squeeze()
        
        # Ensure output and label shapes are compatible for the loss function
        if output.shape != label.shape:
            print(f"Output shape: {output.shape}, Label shape: {label.shape}")
        
        loss = loss_function(output, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i % 50 == 49:
            print(f"Batch {i+1}, Loss: {epoch_loss / (i+1)}")
    
    average_epoch_loss = epoch_loss / len(static_train_dataloader)
    print(f"Average loss for epoch {epoch+1}: {average_epoch_loss}")
    
    if average_epoch_loss < loss_scores[-1]:
        torch.save(static_model.state_dict(), "../other_pickle/Static_Model.pth")
        print("Model saved")
    loss_scores.append(average_epoch_loss)

Epoch 1/2
Batch 50, Loss: 0.10775196806054581
Batch 100, Loss: 0.09472923024047718
Batch 150, Loss: 0.09064534934671269
Batch 200, Loss: 0.08853876835137076
Batch 250, Loss: 0.08805167887432032
Batch 300, Loss: 0.08702061069858083
Batch 350, Loss: 0.08729162992295826
Batch 400, Loss: 0.08752628200982747
Batch 450, Loss: 0.08712476874389594
Batch 500, Loss: 0.08722365264252695
Batch 550, Loss: 0.08729035351475259
Batch 600, Loss: 0.08681666268354615
Batch 650, Loss: 0.08667377470334678
Batch 700, Loss: 0.08624226868232394
Batch 750, Loss: 0.08597234619382214
Batch 800, Loss: 0.08591118572180756
Batch 850, Loss: 0.08609858182644
Batch 900, Loss: 0.08630155578027436
Batch 950, Loss: 0.08613220371188478
Batch 1000, Loss: 0.08597507376754862
Batch 1050, Loss: 0.08602298062781169
Batch 1100, Loss: 0.08613478705194759
Batch 1150, Loss: 0.0860482809452679
Batch 1200, Loss: 0.08603195771924284
Batch 1250, Loss: 0.08583584865879834
Batch 1300, Loss: 0.08582224631120536
Batch 1350, Loss: 0.085648

c:\Users\ficak\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Batch 50, Loss: 0.08341466733226487
Batch 100, Loss: 0.0850143377189131
Batch 150, Loss: 0.08450723371134278
Batch 200, Loss: 0.082964066748668
Batch 250, Loss: 0.08365548972261229
Batch 300, Loss: 0.08410297123576765
Batch 350, Loss: 0.08432670760269825
Batch 400, Loss: 0.085249662832735
Batch 450, Loss: 0.08513972609176604
Batch 500, Loss: 0.08536617976112969
Batch 550, Loss: 0.08575323468577666
Batch 600, Loss: 0.08573773523330117
Batch 650, Loss: 0.08579665911654963
Batch 700, Loss: 0.08581389779596893
Batch 750, Loss: 0.08609370155974287
Batch 800, Loss: 0.08616305208052215
Batch 850, Loss: 0.0863034998942753
Batch 900, Loss: 0.0865207479057329
Batch 950, Loss: 0.0865219340331799
Batch 1000, Loss: 0.08638159780266902
Batch 1050, Loss: 0.0863253317811039
Batch 1100, Loss: 0.0863473679330785
Batch 1150, Loss: 0.08642075101277094
Batch 1200, Loss: 0.08630195078800076
Batch 1250, Loss: 0.08623328953156471
Batch 1300, Loss: 0.0860420839063351
Batch 1350, Loss: 0.08606990241615561
Batch

In [ ]:
static_model.load_state_dict(torch.load("../other_pickle/Static_Model.pth"))
static_model.eval()
for i, batch in list(enumerate(static_train_dataloader))[:10]:
    (input, worthless_input), label = batch
    output = static_model(input).squeeze()
    # loss = loss_function(output, label)
    print(output)

Dynamic model

In [145]:
HIDDEN_SIZE = 200
LAYERS = 8

class DynamicLSTM(nn.Module):

    def __init__(self, hidden_dim, batch_size, layers, input, categories=0):
        super(DynamicLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.layers_num = layers
        
        #input is all the embedding vectors plus all the other variables
        self.lstm = nn.LSTM(input, hidden_dim, num_layers=layers, batch_first=True) 
        self.hidden = (torch.zeros(layers,batch_size,hidden_dim),torch.zeros(layers,batch_size,hidden_dim))
        
        #Squeeeze them into 1 dimension
        if categories > 0:
            self.hidden2label = nn.Linear(hidden_dim, categories)
        else:
            self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, batch_tensor):
        lstm_out, self.hidden = self.lstm(batch_tensor)
        last_timestep_output = lstm_out[:, -1, :]
        sales = self.hidden2label(last_timestep_output)
        return sales
    
    def hidden_reset(self):
        #reset the hidden and cell state after each epoch
        self.hidden = (torch.zeros(self.layers_num,self.batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,self.batch_size,self.hidden_dim))
    def batch_reset(self,batch_size):
        self.hidden = (torch.zeros(self.layers_num,batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,batch_size,self.hidden_dim))
    def flatten_parameters(self):
        self.lstm.flatten_parameters()

dynamic_model = DynamicLSTM(HIDDEN_SIZE, STATIC_BATCH_SIZE, LAYERS, dynamic_size, categories=9)
dynamic_model = dynamic_model.double()

In [156]:
loss_function = nn.CrossEntropyLoss()
loss_scores = [float('inf')]
learning_rate = 0.1
epochs = 1
dynamic_model = dynamic_model.to(device)
optimizer = optim.Adam(dynamic_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    dynamic_model.hidden_reset()
    epoch_loss = 0
    for i, batch in enumerate(dynamic_train_dataloader):
        (input, worthless_input), label = batch
        if input.shape[0] != DYNAMIC_BATCH_SIZE:
            dynamic_model.batch_reset(input.shape[0])
            print("Reset triggered due to batch size mismatch")

        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = dynamic_model(input).squeeze()
        
        # Ensure output and label shapes are compatible for the loss function
        if output.shape != label.shape:
            print(f"Output shape: {output.shape}, Label shape: {label.shape}")
        
        loss = loss_function(output, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i % 10 == 9:
            print(f"Batch {i+1}, Loss: {epoch_loss / (i+1)}")
    
    average_epoch_loss = epoch_loss / len(dynamic_train_dataloader)
    print(f"Average loss for epoch {epoch+1}: {average_epoch_loss}")
    
    if average_epoch_loss < loss_scores[-1]:
        torch.save(dynamic_model.state_dict(), "../other_pickle/Dynamic_Model.pth")
        print("Model saved")
    loss_scores.append(average_epoch_loss)

Epoch 1/1
Batch 10, Loss: 1.319769970832645
Batch 20, Loss: 1.402531977834551
Batch 30, Loss: 1.464933596904132
Batch 40, Loss: 1.4592754595945305
Batch 50, Loss: 1.3818568124136457
Batch 60, Loss: 1.3636398481749858
Batch 70, Loss: 1.3132020866374143
Batch 80, Loss: 1.3108535315241712
Batch 90, Loss: 1.3094457392159218
Batch 100, Loss: 1.3350681369344393
Batch 110, Loss: 1.329348893167229
Batch 120, Loss: 1.3456242269003726
Batch 130, Loss: 1.366603802401159
Batch 140, Loss: 1.4289747565367938
Batch 150, Loss: 1.5224066097331046
Batch 160, Loss: 1.5296784347681953
Batch 170, Loss: 1.5473424931930428
Batch 180, Loss: 1.535560604618781
Batch 190, Loss: 1.5372189313142395
Batch 200, Loss: 1.5588628976616474
Batch 210, Loss: 1.5526475355553524
Batch 220, Loss: 1.5599478513441942
Batch 230, Loss: 1.5665123795053804
Batch 240, Loss: 1.5615688519629691
Batch 250, Loss: 1.5590981741086294
Batch 260, Loss: 1.569677791923514
Batch 270, Loss: 1.5643889242132076
Batch 280, Loss: 1.566394245782407

KeyboardInterrupt: 

In [152]:
dynamic_model.load_state_dict(torch.load("../other_pickle/Dynamic_Model.pth"))
dynamic_model.eval()
for i, batch in list(enumerate(dynamic_train_dataloader))[:100]:
    (input, worthless_input), label = batch
    output = dynamic_model(input).squeeze()
    # loss = loss_function(output, label)
    print(F.softmax(output, dim=1), label)

tensor([[2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01, 8.1895e-02, 9.0988e-03,
         1.2079e-03, 1.2944e-03, 3.6838e-04],
        [2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01, 8.1895e-02, 9.0988e-03,
         1.2079e-03, 1.2944e-03, 3.6838e-04],
        [2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01, 8.1895e-02, 9.0988e-03,
         1.2079e-03, 1.2944e-03, 3.6838e-04],
        [2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01, 8.1895e-02, 9.0988e-03,
         1.2079e-03, 1.2944e-03, 3.6838e-04],
        [2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01, 8.1895e-02, 9.0988e-03,
         1.2079e-03, 1.2944e-03, 3.6838e-04]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=torch.float64)
tensor([[2.4551e-10, 1.5086e-04, 3.8396e-01, 5.2203e-01,

Indicator model